In [13]:
import io
import progressbar
import os
from fractions import Fraction, gcd

In [14]:
def lcm(numbers):
        
        def lcmm(a, b):
            return a * b / gcd(a, b)
        return reduce(lcmm, numbers)

In [15]:
class TrieNode:
    def __init__(self, let = ''):
        self.letter = let
        self.children = {}
        self.count = 1
        self.count_children = 0
    
    def printNode(self):
        print self.letter
        print 'num_childrens: {}, count: {}'.format(len(self.children), self.count)
        
class Trie:
    def __init__(self):
        self.root_node = TrieNode()
        self.deepest_level = 0
        
    #get a word to insert the last letter for ppm
    #updates counts used for predictions
    #updates the deepest level to be used in context size
    def insert(self, word):
        node = self.root_node
        deep = -1
        for i, let in enumerate(word):
            past_node = node
            if node.children.get(let, None) != None:
                node = node.children[let]
                if i == len(word) - 1 and len(word) != 1:
                    node.count += 1
                    past_node.count_children += 1
            else:
                new_node = TrieNode(let)
                node.children[let] = new_node
                node = new_node
                past_node.count_children += 1
            deep += 1
        if deep > self.deepest_level:
            self.deepest_level = deep
    
    #searches for a word if it exists
    #returns 0, 0 if there is no childrens in this level
    #returns the count of the last letter given these letters before it and the total count of other chracters 
    #with the escape character
    def find(self, word):
        node = self.root_node
        i = 0
        while i < len(word) - 1:
            if node.children.get(word[i], None) != None:
                node = node.children[word[i]]
                i += 1
            else:
                break
                
        if i == len(word) - 1:
            if len(node.children) == 0:
                return 0, 0
            if node.children.get(word[i], None) != None:
                count = node.children[word[i]].count
            else:
                count = 0
            cum_count = node.count_children
            cum_count += 1
            return count, cum_count
        return 0, 0
    
    def getChilds(self, word):
        node = self.root_node
        i = 0
        while i < len(word):
            if node.children.get(word[i], None) != None:
                node = node.children[word[i]]
                i += 1
            else:
                break
        if i != len(word):
            return {}
        return node.children.copy()
    
    def printTrie(self, node = None):
        if node == None:
            node = self.root_node
        node.printNode()
        for k, v in node.children.iteritems():
            self.printTrie(v)

In [16]:
class ARIEn:
    def __init__(self,max_len = 31):
        self.max_len = max_len
        self.low = 0
        self.high = 2**max_len - 1
        self.total_bits = 0
        self.bits_count = 0
        
    def encodeOneSymbol (self,lowRange,highRange,totalNum):   
        rangeS = float(self.high - self.low + 1) / float(totalNum)
        self.high = self.low + int(highRange * rangeS) - 1
        self.low = self.low + int(lowRange * rangeS)
        out = ''
        
        
        while (self.low & (1<<(self.max_len-1))) == (self.high & (1<<(self.max_len-1))) or ((self.low & (1<<(self.max_len - 2)) != 0) and ((self.high & (1<<(self.max_len - 2)) == 0))):
            if (self.low & (1<<(self.max_len-1))) == (self.high & (1<<(self.max_len-1))):
                b = int((self.low>>(self.max_len-1)) & 1)
                out += str(b)
                self.total_bits += 1
                self.low = self.low << 1
                self.low = self.low & (2**self.max_len - 1)
                self.high = self.high << 1
                self.high = self.high | 1
                self.high = self.high & (2**self.max_len - 1)
                while self.bits_count > 0:
                    out += str(int(not(b)))
                    self.total_bits += 1
                    self.bits_count -= 1

            if ((self.low & (1<<(self.max_len - 2)) != 0) and ((self.high & (1<<(self.max_len - 2)) == 0))):
                self.bits_count += 1 
                self.low = self.low & ((1<<(self.max_len - 2))-1)
                self.low = self.low << 1
                self.high = self.high | (1<<(self.max_len - 2))
                self.high = self.high << 1
                self.high = self.high | 1
                self.high = self.high & (2**self.max_len - 1)
        return out

class ARIDe:
    def __init__(self,bitStr,max_len = 31):
        self.max_len = max_len
        self.low = 0
        self.high = 2**max_len - 1
        self.bitStr = bitStr
        self.buffer = int(self.bitStr[:max_len],2)
        self.bitNum = max_len

    def decodeOneSymbol (self,counts,totalNum):
        k=0
        while (int(((self.buffer-self.low+1)*totalNum-1)/(self.high-self.low+1)) >= counts[k]):
            k += 1
        
        
        rangeS = float(self.high - self.low + 1) / float(totalNum)
        self.high = self.low + int(counts[k] * rangeS) - 1
        if k != 0:
          self.low = self.low + int(counts[k-1] * rangeS)
        else:
          self.low = self.low

        while ((self.low & (1<<(self.max_len-1))) == (self.high & (1<<(self.max_len-1)))) or ((self.low & (1<<(self.max_len - 2)) != 0) and ((self.high & (1<<(self.max_len - 2)) == 0))):      
            if (self.low & (1<<(self.max_len-1))) == (self.high & (1<<(self.max_len-1))):
                self.low <<= 1
                self.low &= (2**self.max_len - 1)
                self.high <<= 1
                self.high |= 1
                self.high &= (2**self.max_len - 1)
                self.buffer <<= 1
                try:
                  self.buffer |= int(self.bitStr[self.bitNum])
                  self.bitNum += 1           
                except: 
                  pass
                self.buffer &= (2**self.max_len - 1)

            if ((self.low & (1<<(self.max_len - 2)) != 0) and ((self.high & (1<<(self.max_len - 2)) == 0))):
                self.low = self.low & ((1<<(self.max_len - 2))-1)
                self.low = self.low << 1
                self.high = self.high | (1<<(self.max_len - 2))
                self.high = self.high << 1
                self.high = self.high | 1
                self.high = self.high & (2**self.max_len - 1)
                self.buffer ^= (1<<(self.max_len - 2))
                self.buffer <<= 1
                try:
                  self.buffer |= int(self.bitStr[self.bitNum])
                  self.bitNum += 1
                except:
                  pass
                self.buffer &= (2**self.max_len - 1)

        return k

In [141]:
class PPMEncoder:
    def __init__(self, dictionary = 'dict.txt'):
        self.letters_counts={}
        self.file_str = ''
        self.file_path = ''
        self.encoded_path = ''
        self.codes = []
        self.output = ''
        self.trie = Trie()
        self.order = 3
        self.probabilities = {}
        self.total_count = 0
        self.ari_encoder = ARIEn()
        self.base = io.open(dictionary, mode="r", encoding="utf-8").read()
        self.base = self.base.replace('\n','\r\n')
        for let in self.base:
            if self.letters_counts.has_key(let):
                self.letters_counts[let] = self.letters_counts[let] + 1
            else:
                self.letters_counts[let] = 1
                self.codes.append(let)
                self.trie.insert(let)
        self.trie.insert(u'z')
        self.codes.sort()
        self.codes.append(u'z')
        for k in self.codes:
            self.probabilities[k] = [1, False, 0, 0]
        
    def readFile(self, file_path):
        self.file_path = file_path
        self.file_str = io.open(file_path, mode="r", encoding="utf-8").read()
        self.file_str = self.file_str.replace('\n','\r\n')
        
    def updateCounts(self, word):
        i = len(word) - 1
        while i >= 0:
            self.trie.insert(word[i:])
            i -= 1
            
            
    def updateProbabilities(self, word):
        for k in self.codes:
            self.probabilities[k] = [Fraction(1), False, 0, 0]
        calculated_letters = []
        i = 0
        while i < len(word):
            childs = self.trie.getChilds(word[i:len(word)])
            if childs != {}:
                past_counts = 0
                for item in calculated_letters:
                    past_counts += childs[item].count
                for let in self.codes:
                    if not(self.probabilities[let][1]):
                        count, total_count = self.trie.find(word[i: len(word)] + let)
                        total_count -= past_counts
                        if count != 0:
                            self.probabilities[let][0] *= Fraction(count, total_count)
                            calculated_letters.append(let)
                            self.probabilities[let][1] = True
                        else:
                            self.probabilities[let][0] *= Fraction(1, total_count + 1)
            i += 1
        ceil = 0
        numbers = [v[0].denominator for k, v in self.probabilities.iteritems()]
        m = lcm(numbers)
        for k in self.codes:
            self.probabilities[k][0] *= m
            self.probabilities[k][2] = ceil
            ceil += self.probabilities[k][0].numerator
            self.probabilities[k][3] = ceil
        self.total_count = ceil + 1
            
    def initializeProbabilities(self):
        ceil = 0
        node = self.trie.root_node
        self.total_count = 1
        for k in self.codes:
            self.probabilities[k][0] = Fraction(node.children[k].count)
            self.total_count += node.children[k].count
            self.probabilities[k][2] = ceil
            ceil += self.probabilities[k][0].numerator
            self.probabilities[k][3] = ceil

        
    def encode(self):
        i = 0
        self.initializeProbabilities()
        with progressbar.ProgressBar(max_value=len(self.file_str)) as bar:
            while i < len(self.file_str):
                #encode with arithmetic encoder
                self.output += self.ari_encoder.encodeOneSymbol(self.probabilities[self.file_str[i]][2], self.probabilities[self.file_str[i]][3], self.total_count)
                
                #update counts
                self.updateCounts(self.file_str[max(0, i - self.order): i+1])

                #update probabilities
                self.updateProbabilities(self.file_str[max(0, i - self.order): i+1])               
                
                i += 1
                bar.update(i)
            self.output += self.ari_encoder.encodeOneSymbol(self.probabilities[u'z'][2], self.probabilities[u'z'][3], self.total_count)
            self.output += self.ari_encoder.encodeOneSymbol(self.probabilities[u'z'][2], self.probabilities[u'z'][3], self.total_count)
            
    def printEncodedFile(self, file_name = 'encoded_file.bin'):
        byte_array = []
        self.encoded_path = file_name
        i = 0
        while i < len(self.output):
            byte_array.append(int(self.output[i: i+8], 2))
            i += 8
        rest_bits = len(self.output) % 8
        if rest_bits != 0:
            byte_array[len(byte_array) - 1] = byte_array[len(byte_array) - 1] << (8 - rest_bits)
        byte_array = bytearray(byte_array)
        newFile = open(file_name, 'wb')
        newFile.write(byte_array)
        newFile.close() 
        
    def calCompressionRatio(self):
        return os.path.getsize(self.file_path) * 1.0 / os.path.getsize(self.encoded_path)

In [142]:
class PPMDecoder:
    def __init__(self, dictionary = 'dict.txt'):
        self.letters_counts={}
        self.file_str = ''
        self.file_path = ''
        self.codes = []
        self.output = ''
        self.decoded = ''
        self.decoded_path = ''
        self.encoded_string = ''
        self.bin_path = ''
        self.trie = Trie()
        self.order = 3
        self.probabilities = {}
        self.total_count = 0
        self.base = io.open(dictionary, mode="r", encoding="utf-8").read()
        self.base = self.base.replace('\n','\r\n')
        for let in self.base:
            if self.letters_counts.has_key(let):
                self.letters_counts[let] = self.letters_counts[let] + 1
            else:
                self.letters_counts[let] = 1
                self.codes.append(let)
                self.trie.insert(let)
        self.trie.insert(u'z')
        self.codes.sort()
        self.codes.append(u'z')
        for k in self.codes:
            self.probabilities[k] = [1, False, 0, 0]

    def updateCounts(self, word):
        i = len(word) - 1
        while i >= 0:
            self.trie.insert(word[i:])
            i -= 1
            
            
    def updateProbabilities(self, word):
        for k in self.codes:
            self.probabilities[k] = [Fraction(1), False, 0, 0]
        calculated_letters = []
        i = 0
        while i < len(word):
            childs = self.trie.getChilds(word[i:len(word)])
            if childs != {}:
                past_counts = 0
                for item in calculated_letters:
                    past_counts += childs[item].count
                for let in self.codes:
                    if not(self.probabilities[let][1]):
                        count, total_count = self.trie.find(word[i: len(word)] + let)
                        total_count -= past_counts
                        if count != 0:
                            self.probabilities[let][0] *= Fraction(count, total_count)
                            calculated_letters.append(let)
                            self.probabilities[let][1] = True
                        else:
                            self.probabilities[let][0] *= Fraction(1, total_count + 1)
            i += 1
        ceil = 0
        numbers = [v[0].denominator for k, v in self.probabilities.iteritems()]
        m = lcm(numbers)
        for k in self.codes:
            self.probabilities[k][0] *= m
            self.probabilities[k][2] = ceil
            ceil += self.probabilities[k][0].numerator
            self.probabilities[k][3] = ceil
        self.total_count = ceil + 1
            
    def initializeProbabilities(self):
        ceil = 0
        node = self.trie.root_node
        self.total_count = 1
        for k in self.codes:
            self.probabilities[k][0] = Fraction(node.children[k].count)
            self.total_count += node.children[k].count
            self.probabilities[k][2] = ceil
            ceil += self.probabilities[k][0].numerator
            self.probabilities[k][3] = ceil  


    def decode(self):
        i = 0
        self.initializeProbabilities()
        self.ari_decoder = ARIDe(self.output)
        with progressbar.ProgressBar(max_value=len(self.output)) as bar:
            while True:
                #decode with aritthmetic encoding
                k = self.ari_decoder.decodeOneSymbol([self.probabilities[let][3] for let in self.codes], self.total_count)
                k = self.codes[k]
                if k == 'z':
                    break
                self.decoded += k

                #update counts
                self.updateCounts(self.decoded[max(0, i - self.order): i+1])

                #update probabilities
                self.updateProbabilities(self.decoded[max(0, i - self.order): i+1])
                i += 1
                bar.update(self.ari_decoder.bitNum)
            
    def readEncodedFile(self, file_name = 'encoded_file.bin'):
        self.bin_path = file_name
        f = open(file_name, 'rb')
        byte_array = f.read()
        self.encoded_string = ''
        i = 0
        with progressbar.ProgressBar(max_value=len(byte_array)) as bar:
            for byte in byte_array:
                self.encoded_string += bin(ord(byte))[2:].zfill(8)
                i += 1
                bar.update(i)
        f.close()
        self.output = self.encoded_string
        
    def printDecodedFile(self, decoded_path = 'decoded.tsv'):
        self.decoded_path = decoded_path
        f= io.open(decoded_path, mode="w", encoding="utf-8")
        f.write(self.decoded)
        f.close()

In [143]:
def Encode(file_path, dictionary = 'dict.txt', encoded_path = 'encoded_file.bin'):
    ppmen = PPMEncoder(dictionary)
    ppmen.readFile(file_path)
    ppmen.encode()
    ppmen.printEncodedFile(encoded_path)
    return ppmen

In [144]:
def Decode(file_path, dictionary = 'dict.txt', decoded_path = 'decoded.tsv'):
    ppmde = PPMDecoder(dictionary)
    ppmde.readEncodedFile(file_path)
    ppmde.decode()
    ppmde.printDecodedFile(decoded_path)
    return ppmde

In [145]:
def CheckTwoStrings(s1, s2):
    return s1 == s2

In [150]:
en = Encode('omar.tsv')

100% (20603 of 20603) |##################| Elapsed Time: 0:01:27 Time:  0:01:27


In [151]:
de = Decode('encoded_file.bin')

100% (10039 of 10039) |##################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (80312 of 80312) |##################| Elapsed Time: 0:01:26 Time:  0:01:26


In [152]:
print CheckTwoStrings(en.file_str, de.decoded)

True


In [153]:
print en.calCompressionRatio()

3.63203506325
